In [1]:
import pandas as pd
import openpyxl
import os
import re

# 读取Excel文件
df = pd.read_excel('../zuoyue.xlsx')

journal_zhuoyue = {}

mylist = ['2018','2017','2016','2015','2014','2013','2012','2011','2010']
#mylist = ['2024','2023','2022','2021','2020','2019']
#mylist = ['test']
#mylist = ['1wmore']

def get_text_before_paren(text):
    # 正则表达式匹配第一个括号或冒号前的字符
    match = re.search(r'(.*?)(\(|\:|\-)', text)
    if match:
        return match.group(1).strip()
    else:
        return text.strip()

def list_excel_files(directory):
    """
    遍历指定目录下所有以.xlsx结尾的文件，并返回文件路径列表。
    :return: 文件路径列表
    """
    return [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(directory)) for f in fn if f.endswith('.xlsx')]
        
for index, row in df.iterrows():
    key = row[df.columns[1]].title()    
    key = get_text_before_paren(key)
    key = re.sub(r'^The ', "", key)
    key = re.sub(r'/', " ", key)
    key = re.sub(r'\. ', " ", key)
    key = re.sub(r'\, ', " ", key)
    key = re.sub(r'  ', " ", key)
    journal_zhuoyue[key] = row[df.columns[2]]

for i in mylist:
    directory = './paper_donload/' + i + '/'
    excel_files = list_excel_files(directory)
    for path in excel_files:
        wb = openpyxl.load_workbook(path)
        ws = wb["Sheet"]

        for cur_row in range(2, ws.max_row+1):
            j_name = ws.cell(row=cur_row, column=6).value
            #j_short = ws.cell(row=cur_row, column=5).value
            try:
                j_name = j_name.title()
                j_name = get_text_before_paren(j_name)
                j_name = re.sub(r'^The ', "", j_name)
                j_name = re.sub(r'/', " ", j_name)
                j_name = re.sub(r'\. ', " ", j_name)
                j_name = re.sub(r'\, ', " ", j_name)
                j_name = re.sub(r'arkinson\'s', "arkinsons", j_name)
                j_name = re.sub(r'omen\'s', "omens", j_name)
                if j_name in journal_zhuoyue:
                    ws.cell(row=cur_row, column=27).value = journal_zhuoyue[j_name]
                else:
                    ws.cell(row=cur_row, column=27).value = 'NA'
            except:     
                ws.cell(row=cur_row, column=27).value = 'NA'

        ws['AA1'] = '卓越期刊类别'

        wb.save(path)
